# Company Entity Resolution: Matching Business Names Across Datasets

## The Challenge

In M&A due diligence, investment research, and B2B sales, you constantly need to match company names across different data sources. The problem: companies appear differently in each system:

| Source A | Source B | Same Company? |
|----------|----------|---------------|
| Apple Inc. | Apple, Inc | Yes |
| Goldman Sachs Group Inc | Goldman & Sachs | Yes |
| International Business Machines Corporation | IBM Corp. | Yes |
| Berkshire Hathaway Inc | Berkshire Hathaway Insurance | Maybe? |

Legal entity suffixes (Inc., Corp., LLC, Ltd.), punctuation variations, and abbreviations make simple string matching fail.

## What You'll Learn

1. **Normalizing company names**: Removing legal suffixes and standardizing format
2. **Batch matching**: Efficiently comparing large company lists
3. **Threshold tuning**: Balancing precision and recall for business data
4. **Handling subsidiaries**: When "Alphabet" should match "Google"

## Setup

### Dataset: SEC EDGAR Company Names

Download the SEC company list:
- **Kaggle**: https://www.kaggle.com/datasets/caesarlupum/gov-names
- Contains 800K+ registered company names with CIK identifiers

In [ ]:
import fuzzyrust as fr
import csv
import re
import time
from pathlib import Path

print("FuzzyRust loaded for company matching")

In [ ]:
# Corporate name normalization
LEGAL_SUFFIXES = [
    # Common suffixes (order matters - longer first)
    "incorporated", "corporation", "limited liability company",
    "limited partnership", "limited", "company",
    "inc.", "inc", "corp.", "corp", "co.", "co",
    "llc", "l.l.c.", "llp", "l.l.p.", "lp", "l.p.",
    "ltd.", "ltd", "plc", "p.l.c.",
    "n.a.", "na", "& co", "& company",
    "group", "holdings", "enterprises",
]

def normalize_company_name(name: str) -> str:
    """
    Normalize a company name for matching.
    
    1. Convert to lowercase
    2. Remove punctuation (except &)
    3. Remove legal suffixes
    4. Normalize whitespace
    """
    if not name:
        return ""
    
    # Lowercase
    result = name.lower().strip()
    
    # Replace common punctuation
    result = result.replace(",", " ").replace(".", " ")
    result = result.replace("-", " ").replace("'", "")
    
    # Remove legal suffixes
    for suffix in LEGAL_SUFFIXES:
        # Match at end of string or before punctuation/space
        pattern = r"\b" + re.escape(suffix) + r"\b"
        result = re.sub(pattern, "", result)
    
    # Normalize whitespace
    result = " ".join(result.split())
    
    return result.strip()

# Test normalization
test_names = [
    "Apple Inc.",
    "APPLE, INC",
    "Goldman Sachs Group, Inc.",
    "International Business Machines Corporation",
    "Berkshire Hathaway Inc",
    "JPMorgan Chase & Co.",
]

print("Company Name Normalization")
print("=" * 70)
print(f"{'Original':<45} {'Normalized'}")
print("-" * 70)

for name in test_names:
    normalized = normalize_company_name(name)
    print(f"{name:<45} {normalized}")

In [ ]:
# Sample company data (simulating SEC EDGAR format)
# In production, load from: https://www.kaggle.com/datasets/caesarlupum/gov-names

SEC_COMPANIES = [
    # Tech giants
    {"cik": "0000320193", "name": "APPLE INC"},
    {"cik": "0001652044", "name": "ALPHABET INC"},
    {"cik": "0000789019", "name": "MICROSOFT CORPORATION"},
    {"cik": "0001018724", "name": "AMAZON COM INC"},
    {"cik": "0001326801", "name": "META PLATFORMS INC"},
    {"cik": "0001318605", "name": "TESLA INC"},
    {"cik": "0001045810", "name": "NVIDIA CORPORATION"},
    
    # Financial services
    {"cik": "0000886982", "name": "JPMORGAN CHASE & CO"},
    {"cik": "0000070858", "name": "BANK OF AMERICA CORPORATION"},
    {"cik": "0000831001", "name": "CITIGROUP INC"},
    {"cik": "0000769993", "name": "WELLS FARGO & COMPANY"},
    {"cik": "0000886946", "name": "GOLDMAN SACHS GROUP INC"},
    {"cik": "0000895421", "name": "MORGAN STANLEY"},
    
    # Healthcare
    {"cik": "0000731766", "name": "UNITEDHEALTH GROUP INCORPORATED"},
    {"cik": "0000310158", "name": "JOHNSON & JOHNSON"},
    {"cik": "0000078003", "name": "PFIZER INC"},
    {"cik": "0000021344", "name": "COCA COLA CO"},
    
    # Industrial
    {"cik": "0000051143", "name": "INTERNATIONAL BUSINESS MACHINES CORP"},
    {"cik": "0000040545", "name": "GENERAL ELECTRIC CO"},
    {"cik": "0000012927", "name": "BOEING CO"},
    {"cik": "0000093410", "name": "CHEVRON CORPORATION"},
    {"cik": "0000034088", "name": "EXXON MOBIL CORPORATION"},
    
    # Retail
    {"cik": "0000104169", "name": "WALMART INC"},
    {"cik": "0001058290", "name": "HOME DEPOT INC"},
    {"cik": "0000027419", "name": "COSTCO WHOLESALE CORPORATION"},
    
    # Conglomerates
    {"cik": "0001067983", "name": "BERKSHIRE HATHAWAY INC"},
    {"cik": "0000051143", "name": "3M COMPANY"},
]

print(f"Loaded {len(SEC_COMPANIES)} companies from SEC data")

## Building a Company Matcher

Let's create a reusable company matching system that:
1. Normalizes company names
2. Uses an index for fast lookup
3. Returns matched companies with confidence scores

In [ ]:
class CompanyMatcher:
    """
    Match company names against a reference database.
    
    Uses normalization + N-gram indexing + Jaro-Winkler scoring.
    """
    
    def __init__(self):
        self.index = fr.NgramIndex(ngram_size=3)
        self.companies = {}  # normalized_name -> original record
        self.original_names = {}  # id -> original name
    
    def add_companies(self, companies: list[dict]):
        """Add companies to the matching index."""
        for i, company in enumerate(companies):
            original = company["name"]
            normalized = normalize_company_name(original)
            
            # Index the normalized name
            self.index.add_with_data(normalized, i)
            self.companies[normalized] = company
            self.original_names[i] = original
        
        print(f"Indexed {len(companies)} companies")
    
    def match(self, query: str, limit: int = 5, min_similarity: float = 0.7) -> list[dict]:
        """
        Find matching companies for a query.
        
        Returns list of dicts with: original_name, normalized_name, score, cik
        """
        # Normalize the query
        normalized_query = normalize_company_name(query)
        
        if not normalized_query:
            return []
        
        # Search the index
        results = self.index.search(
            normalized_query,
            algorithm="jaro_winkler",
            min_similarity=min_similarity,
            limit=limit
        )
        
        # Format results
        matches = []
        for r in results:
            company = self.companies.get(r.text, {})
            matches.append({
                "query": query,
                "normalized_query": normalized_query,
                "matched_name": self.original_names.get(r.data, r.text),
                "normalized_match": r.text,
                "score": r.score,
                "cik": company.get("cik", "N/A"),
            })
        
        return matches
    
    def match_batch(self, queries: list[str], limit: int = 3, min_similarity: float = 0.7) -> dict:
        """
        Match multiple company names efficiently.
        
        Returns dict mapping query -> list of matches
        """
        return {q: self.match(q, limit, min_similarity) for q in queries}


# Create and populate the matcher
matcher = CompanyMatcher()
matcher.add_companies(SEC_COMPANIES)

In [ ]:
# Test with realistic queries
test_queries = [
    "Apple Inc.",                    # Exact match (different punctuation)
    "apple",                         # Minimal query
    "Goldman Sachs",                 # Without "Group Inc"
    "JP Morgan",                     # Space variation
    "IBM",                           # Abbreviation (won't match well)
    "International Business Machines",  # Full name
    "Berkshire Hathaway",            # Common name
    "Tesla Motors",                  # Old name variation
    "Amazon.com Inc",                # With .com
    "Wells Fargo Bank",              # With business unit
]

print("Company Matching Results")
print("=" * 80)

for query in test_queries:
    matches = matcher.match(query, limit=2, min_similarity=0.6)
    
    print(f"\n🔍 Query: '{query}'")
    print(f"   Normalized: '{normalize_company_name(query)}'")
    
    if matches:
        for m in matches:
            print(f"   [{m['score']:.3f}] {m['matched_name']} (CIK: {m['cik']})")
    else:
        print("   No matches found")

## Handling Abbreviations and Aliases

Notice that "IBM" doesn't match "International Business Machines" because they share no characters. For this, we need an alias table:

In [ ]:
# Common company aliases/abbreviations
COMPANY_ALIASES = {
    "ibm": "international business machines",
    "ge": "general electric",
    "gm": "general motors",
    "att": "at&t",
    "hp": "hewlett packard",
    "pg": "procter gamble",
    "jnj": "johnson johnson",
    "ko": "coca cola",
    "jpm": "jpmorgan chase",
    "bofa": "bank of america",
    "wfc": "wells fargo",
    "gs": "goldman sachs",
    "ms": "morgan stanley",
    "amzn": "amazon",
    "msft": "microsoft",
    "aapl": "apple",
    "goog": "alphabet",
    "google": "alphabet",
    "fb": "meta platforms",
    "facebook": "meta platforms",
    "tsla": "tesla",
}

class EnhancedCompanyMatcher(CompanyMatcher):
    """Company matcher with alias support."""
    
    def __init__(self, aliases: dict = None):
        super().__init__()
        self.aliases = aliases or {}
    
    def expand_query(self, query: str) -> list[str]:
        """Expand a query using aliases."""
        normalized = normalize_company_name(query)
        expansions = [normalized]
        
        # Check if the entire query is an alias
        if normalized in self.aliases:
            expansions.append(self.aliases[normalized])
        
        # Check each word for aliases
        words = normalized.split()
        for word in words:
            if word in self.aliases:
                expanded = normalized.replace(word, self.aliases[word])
                if expanded not in expansions:
                    expansions.append(expanded)
        
        return expansions
    
    def match(self, query: str, limit: int = 5, min_similarity: float = 0.7) -> list[dict]:
        """Match with alias expansion."""
        all_matches = []
        seen = set()
        
        for expanded_query in self.expand_query(query):
            matches = super().match(expanded_query, limit, min_similarity)
            for m in matches:
                key = m['cik']
                if key not in seen:
                    seen.add(key)
                    m['expanded_from'] = expanded_query if expanded_query != normalize_company_name(query) else None
                    all_matches.append(m)
        
        # Sort by score and limit
        all_matches.sort(key=lambda x: x['score'], reverse=True)
        return all_matches[:limit]


# Create enhanced matcher
enhanced_matcher = EnhancedCompanyMatcher(aliases=COMPANY_ALIASES)
enhanced_matcher.add_companies(SEC_COMPANIES)

# Test abbreviations
abbreviation_queries = ["IBM", "GE", "JPM", "Google", "Facebook"]

print("\nAbbreviation Matching with Alias Expansion")
print("=" * 70)

for query in abbreviation_queries:
    matches = enhanced_matcher.match(query, limit=1, min_similarity=0.6)
    
    print(f"\n🔍 '{query}'")
    if matches:
        m = matches[0]
        expansion_note = f" (via '{m['expanded_from']}')" if m.get('expanded_from') else ""
        print(f"   → {m['matched_name']} [{m['score']:.3f}]{expansion_note}")
    else:
        print("   No match found")

## Batch Processing for Large-Scale Matching

When matching thousands of companies (e.g., cross-referencing a portfolio against SEC data), use batch processing:

In [ ]:
# Simulate a portfolio to match
portfolio_companies = [
    "Apple Inc",
    "Microsoft Corp.",
    "Amazon.com, Inc.",
    "Alphabet (Google)",
    "Tesla Motors Inc",
    "JPMorgan Chase",
    "Bank of America Corp",
    "Berkshire Hathaway",
    "Johnson and Johnson",
    "Procter & Gamble",  # Not in our sample SEC data
    "Unknown Corp XYZ",  # Definitely not in data
]

print(f"Matching {len(portfolio_companies)} portfolio companies...")
print("=" * 70)

start = time.time()
results = enhanced_matcher.match_batch(portfolio_companies, limit=1, min_similarity=0.7)
elapsed = time.time() - start

matched = 0
unmatched = []

for query, matches in results.items():
    if matches:
        matched += 1
        m = matches[0]
        status = "✓" if m['score'] >= 0.85 else "~"
        print(f"{status} {query:<30} → {m['matched_name'][:30]:<30} [{m['score']:.2f}]")
    else:
        unmatched.append(query)
        print(f"✗ {query:<30} → No match found")

print(f"\nSummary:")
print(f"  Matched: {matched}/{len(portfolio_companies)} ({100*matched/len(portfolio_companies):.0f}%)")
print(f"  Unmatched: {unmatched}")
print(f"  Time: {elapsed*1000:.1f} ms")

## Finding Duplicates Within a Dataset

Sometimes you need to find duplicate companies within the same dataset (e.g., after merging multiple sources):

In [ ]:
# Dataset with duplicates
merged_list = [
    "Apple Inc.",
    "APPLE INC",
    "Apple, Inc",
    "Microsoft Corporation",
    "Microsoft Corp.",
    "MSFT Corporation",  # Ticker confusion
    "Amazon.com Inc.",
    "Amazon Inc",
    "AMAZON COM INC",
    "Goldman Sachs Group",
    "The Goldman Sachs Group Inc",
    "Berkshire Hathaway",
    "Tesla Inc",
    "General Electric",  # Unique
    "Boeing Company",    # Unique
]

# Normalize before deduplication
normalized_list = [normalize_company_name(c) for c in merged_list]

print("Finding Duplicate Companies")
print("=" * 60)

result = fr.find_duplicates(
    normalized_list,
    algorithm="jaro_winkler",
    min_similarity=0.85,
    normalize="none"  # Already normalized
)

print(f"Found {len(result.groups)} duplicate groups")
print(f"Unique companies: {len(result.unique)}")

# Map back to original names
norm_to_orig = {normalize_company_name(c): c for c in merged_list}

print("\nDuplicate Groups:")
for i, group in enumerate(result.groups, 1):
    print(f"\n  Group {i}:")
    for norm_name in group:
        orig = norm_to_orig.get(norm_name, norm_name)
        print(f"    - {orig}")

## Summary

### Key Takeaways

1. **Normalize aggressively**: Remove legal suffixes, punctuation, and standardize case before matching.

2. **Use alias tables for abbreviations**: "IBM" won't fuzzy-match to "International Business Machines"—you need explicit mappings.

3. **Threshold guidelines for companies**:
   - 0.90+: Very high confidence (likely exact match)
   - 0.80-0.89: Good confidence (minor variations)
   - 0.70-0.79: Moderate confidence (review recommended)
   - Below 0.70: Low confidence (likely false positive)

4. **Handle subsidiaries carefully**: "Goldman Sachs Asset Management" is not the same entity as "Goldman Sachs Group Inc" for some purposes.

### When to Use This Approach

✅ **Good for:**
- M&A due diligence
- Investment portfolio matching
- Vendor/supplier deduplication
- Cross-referencing business databases

❌ **Consider alternatives for:**
- Legal entity verification (use official registries)
- Subsidiaries/parent matching (use corporate hierarchy data)
- International companies (different naming conventions)